In [111]:
import torch
import pandas as pd

class IrisDataset(torch.utils.data.Dataset):
  def __init__(self):
    self.labels = pd.get_dummies(pd.read_csv('./datasets/iris_flower.csv')['Species']) \
      .astype('float32').to_numpy()

    self.data = pd.read_csv('./datasets/iris_flower.csv') \
      .drop('Species', axis=1) \
      .drop('Id', axis=1) \
      .astype('float32').to_numpy()
    
  def __len__(self):
    return len(self.data)

  def __getitem__(self, idx):
    return self.data[idx], self.labels[idx]

dataset = IrisDataset()
train, test = torch.utils.data.random_split(dataset, [120, 30])

train_loader = torch.utils.data.DataLoader(train, batch_size=32, shuffle=True)
test_loader = torch.utils.data.DataLoader(test, batch_size=32, shuffle=True)

In [112]:
class IrisNeuralNetwork(torch.nn.Module):
  def __init__(self):
    super().__init__()

    self.flatten = torch.nn.Flatten()
    self.linear_relu_stack = torch.nn.Sequential(
      torch.nn.Linear(4, 8),
      torch.nn.ReLU(),
      torch.nn.Linear(8, 8),
      torch.nn.ReLU(),
      torch.nn.Linear(8, 3),
      torch.nn.Softmax(dim=1)
    )

  def forward(self, x):
    x = self.flatten(x)
    logits = self.linear_relu_stack(x)
    return logits
  
model = IrisNeuralNetwork().to("cpu")
print(model)

IrisNeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=4, out_features=8, bias=True)
    (1): ReLU()
    (2): Linear(in_features=8, out_features=8, bias=True)
    (3): ReLU()
    (4): Linear(in_features=8, out_features=3, bias=True)
    (5): Softmax(dim=1)
  )
)


In [113]:
criterion = torch.nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

for epoch in range(1000):
  for i, (inputs, targets) in enumerate(train_loader):
    optimizer.zero_grad()
    outputs = model(inputs)
    loss = criterion(outputs, targets)
    loss.backward()
    optimizer.step()
    if epoch % 100 == 0:
      print('Epoch: %d, Loss: %.3f' % (epoch, loss.item()))

Epoch: 0, Loss: 0.233
Epoch: 0, Loss: 0.225
Epoch: 0, Loss: 0.233
Epoch: 0, Loss: 0.232
Epoch: 100, Loss: 0.127
Epoch: 100, Loss: 0.121
Epoch: 100, Loss: 0.119
Epoch: 100, Loss: 0.152
Epoch: 200, Loss: 0.029
Epoch: 200, Loss: 0.034
Epoch: 200, Loss: 0.015
Epoch: 200, Loss: 0.016
Epoch: 300, Loss: 0.007
Epoch: 300, Loss: 0.037
Epoch: 300, Loss: 0.014
Epoch: 300, Loss: 0.001
Epoch: 400, Loss: 0.017
Epoch: 400, Loss: 0.004
Epoch: 400, Loss: 0.007
Epoch: 400, Loss: 0.042
Epoch: 500, Loss: 0.008
Epoch: 500, Loss: 0.002
Epoch: 500, Loss: 0.012
Epoch: 500, Loss: 0.038
Epoch: 600, Loss: 0.002
Epoch: 600, Loss: 0.032
Epoch: 600, Loss: 0.005
Epoch: 600, Loss: 0.014
Epoch: 700, Loss: 0.007
Epoch: 700, Loss: 0.005
Epoch: 700, Loss: 0.030
Epoch: 700, Loss: 0.006
Epoch: 800, Loss: 0.028
Epoch: 800, Loss: 0.001
Epoch: 800, Loss: 0.015
Epoch: 800, Loss: 0.003
Epoch: 900, Loss: 0.002
Epoch: 900, Loss: 0.026
Epoch: 900, Loss: 0.010
Epoch: 900, Loss: 0.009


In [161]:
model.eval()  # Set the model to evaluation mode
correct = 0
total = 0

with torch.no_grad():
  for inputs, targets in test_loader:
    inputs, targets = inputs.to("cpu"), targets.to("cpu")
    
    outputs = model(inputs)
    _, predicted = torch.max(outputs.data, 1)
    
    # Convert one-hot encoded targets to class indices
    target_class = torch.argmax(targets, dim=1)
    print(predicted, target_class)
    
    total += targets.size(0)
    correct += (predicted == target_class).sum().item()

test_accuracy = 100 * correct / total
print(f"Test Accuracy: {test_accuracy:.2f}%")

tensor([2, 0, 1, 1, 2, 1, 1, 0, 2, 2, 2, 2, 1, 2, 0, 2, 2, 2, 0, 0, 2, 1, 0, 1,
        1, 1, 0, 0, 2, 2]) tensor([2, 0, 1, 1, 2, 1, 1, 0, 2, 2, 2, 2, 1, 2, 0, 2, 2, 2, 0, 0, 2, 1, 0, 1,
        1, 1, 0, 0, 2, 2])
Test Accuracy: 100.00%
